In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import fbeta_score

In [2]:
df_train = pd.read_csv("../data/train_new.csv", sep="|")
df_val = pd.read_csv("../data/val_new.csv", sep="|")

In [24]:
def dmc_score_function(label, y_pred):
    loss = 0.0
    for index, item in enumerate(label):
        if label[index] == 0: 
            if y_pred[index] == 0:
                loss += 0.0
            else:
                loss -= 25.0
        else:
            if y_pred[index] == 0:
                loss -= 5.0
            else:
                loss += 5.0
    return loss

def calculate_metrics(x, y, model, tp="scikit"):
    scores = {}
    pred = None
    if tp is "scikit":
        scores['acc'] = model.score(x,y)
        pred = model.predict(x)
    elif tp is "keras":
        pred = model.predict_classes(x)
    scores['dmc'] = dmc_score_function(pred,y)
    scores['f2'] = fbeta_score(pred, y, 2, average='binary')
    return scores


In [5]:
df_wo_frauds_balanced = df_train[df_train.fraud!=1].sample(n=df_train[df_train.fraud==1].count()[0])
df_50_50 = df_wo_frauds_balanced.append(df_train[df_train.fraud==1],ignore_index=True).sample(frac=1)
df_50_50.count()
#Validation dataset
df_val_y = df_val.fraud
df_val_x = df_val.drop(['fraud'], axis=1)

#Train balanced
df_train_balanced_y = df_50_50.fraud
df_train_balanced_x = df_50_50.drop(['fraud'], axis=1)

#Train unbalanced
df_train_unbalanced_y = df_train.fraud
df_train_unbalanced_x = df_train.drop(['fraud'], axis=1)

## Logistic Regression 
`class sklearn.linear_model.LogisticRegression(penalty=’l2’, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver=’warn’, max_iter=100, multi_class=’warn’, verbose=0, warm_start=False, n_jobs=None)`

In [6]:
res = dict()

log_reg_balanced = LogisticRegression().fit(df_train_balanced_x, df_train_balanced_y)
log_reg_unbalanced = LogisticRegression().fit(df_train_unbalanced_x, df_train_unbalanced_y)

res['logistic regression balanced'] = calculate_metrics(df_val_x, df_val_y, log_reg_balanced)
res['logistic regression unbalanced'] = calculate_metrics(df_val_x, df_val_y, log_reg_unbalanced)
print(res)

{'logistic regression unbalanced': {'dmc': -270.0, 'f2': 0.632183908045977, 'acc': 0.9547872340425532}, 'logistic regression balanced': {'dmc': -105.0, 'f2': 0.3951890034364261, 'acc': 0.8829787234042553}}


/home/lukas/Projects/dmc2019/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/lukas/Projects/dmc2019/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## Linear Discriminant Analysis
`class sklearn.discriminant_analysis.LinearDiscriminantAnalysis(solver=’svd’, shrinkage=None, priors=None, n_components=None, store_covariance=False, tol=0.0001)`

In [7]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda_balanced = LinearDiscriminantAnalysis().fit(df_train_balanced_x, df_train_balanced_y)
lda_unbalanced = LinearDiscriminantAnalysis().fit(df_train_unbalanced_x, df_train_unbalanced_y)
res['lda balanced'] = calculate_metrics(df_val_x, df_val_y, lda_balanced)
res['lda unbalanced'] = calculate_metrics(df_val_x, df_val_y, lda_unbalanced)

## KNN 
`class sklearn.neighbors.KNeighborsClassifier(n_neighbors=5, weights=’uniform’, algorithm=’auto’, leaf_size=30, p=2, metric=’minkowski’, metric_params=None, n_jobs=None, **kwargs)`

In [8]:
from sklearn.neighbors import KNeighborsClassifier
knn_balanced = KNeighborsClassifier().fit(df_train_balanced_x, df_train_balanced_y)
knn_unbalanced = KNeighborsClassifier().fit(df_train_unbalanced_x, df_train_unbalanced_y)
res['knn balanced'] = calculate_metrics(df_val_x, df_val_y, knn_balanced)
res['knn unbalanced'] = calculate_metrics(df_val_x, df_val_y, knn_unbalanced)

## Support Vector Classifier
`class sklearn.svm.SVC(C=1.0, kernel=’rbf’, degree=3, gamma=’auto_deprecated’, coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape=’ovr’, random_state=None)`

In [9]:
from sklearn.svm import SVC
svc_balanced = SVC().fit(df_train_balanced_x, df_train_balanced_y)
svc_unbalanced = SVC().fit(df_train_unbalanced_x, df_train_unbalanced_y)
res['svc balanced'] = calculate_metrics(df_val_x, df_val_y, svc_balanced)
res['svc unbalanced'] = calculate_metrics(df_val_x, df_val_y, svc_unbalanced)


/home/lukas/Projects/dmc2019/venv/lib/python3.5/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/lukas/Projects/dmc2019/venv/lib/python3.5/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/lukas/Projects/dmc2019/venv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


In [10]:
from sklearn.ensemble import RandomForestClassifier
rfc_balanced = SVC().fit(df_train_balanced_x, df_train_balanced_y)
rfc_unbalanced = SVC().fit(df_train_unbalanced_x, df_train_unbalanced_y)
res['rfc balanced'] = calculate_metrics(df_val_x, df_val_y, rfc_balanced)
res['rfc unbalanced'] = calculate_metrics(df_val_x, df_val_y, rfc_unbalanced)

/home/lukas/Projects/dmc2019/venv/lib/python3.5/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/lukas/Projects/dmc2019/venv/lib/python3.5/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/lukas/Projects/dmc2019/venv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


## XGBoost 
[XGBoost Webseite](https://xgboost.readthedocs.io/en/latest/index.html)

In [11]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
#params = param = {'max_depth':3, 'eta':1, 'gamma': 1.0, 'min_child_weight' : 1, 'objective':'binary:logistic' }
xgb_balanced = XGBClassifier().fit(df_train_balanced_x,df_train_balanced_y)
xgb_unbalanced = XGBClassifier().fit(df_train_unbalanced_x,df_train_unbalanced_y)
#bst = xgb.train(param, xgb_train_x_balanced, 3)
res['xgb balanced'] = calculate_metrics(df_val_x, df_val_y, xgb_balanced)
res['xgb unbalanced'] = calculate_metrics(df_val_x, df_val_y, xgb_unbalanced)

In [15]:
import numpy as np
np.array(df_val_x).shape

(376, 9)

In [17]:
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import fbeta_score
from keras.models import Sequential
from keras.layers import Dense, Activation

df_train = pd.read_csv("../data/train_new.csv", sep="|")
df_val = pd.read_csv("../data/val_new.csv", sep="|")

# Data Preparation
df_wo_frauds_balanced = df_train[df_train.fraud != 1].sample(n=df_train[df_train.fraud == 1].count()[0])
df_50_50 = df_wo_frauds_balanced.append(df_train[df_train.fraud == 1], ignore_index=True).sample(frac=1)
df_50_50.count()

# Validation dataset
df_val_y = df_val.fraud
df_val_x = df_val.drop(['fraud'], axis=1)

# Train balanced
df_train_balanced_y = df_50_50.fraud
df_train_balanced_x = df_50_50.drop(['fraud'], axis=1)

# Train unbalanced
df_train_unbalanced_y = np.array(df_train.fraud)
df_train_unbalanced_x = np.array(df_train.drop(['fraud'], axis=1))

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=9))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(df_train_unbalanced_x, df_train_unbalanced_y, epochs=15, batch_size=128)

Epoch 1/100
1503/1503 [==============================] - 0s 103us/step - loss: 1.4070 - acc: 0.7911
Epoch 2/100
1503/1503 [==============================] - 0s 22us/step - loss: 0.9879 - acc: 0.8789
Epoch 3/100
1503/1503 [==============================] - 0s 21us/step - loss: 0.8967 - acc: 0.9175
Epoch 4/100
1503/1503 [==============================] - 0s 21us/step - loss: 0.8581 - acc: 0.9335
Epoch 5/100
1503/1503 [==============================] - 0s 19us/step - loss: 1.0309 - acc: 0.8629
Epoch 6/100
1503/1503 [==============================] - 0s 21us/step - loss: 0.8042 - acc: 0.9441
Epoch 7/100
1503/1503 [==============================] - 0s 22us/step - loss: 0.7185 - acc: 0.9348
Epoch 8/100
1503/1503 [==============================] - 0s 21us/step - loss: 1.0343 - acc: 0.8689
Epoch 9/100
1503/1503 [==============================] - 0s 21us/step - loss: 0.7940 - acc: 0.9461
Epoch 10/100
1503/1503 [==============================] - 0s 22us/step - loss: 0.6980 - acc: 0.9381
Epoch 11

1503/1503 [==============================] - 0s 19us/step - loss: 0.7034 - acc: 0.9228
Epoch 84/100
1503/1503 [==============================] - 0s 21us/step - loss: 0.6240 - acc: 0.9022
Epoch 85/100
1503/1503 [==============================] - 0s 23us/step - loss: 0.7658 - acc: 0.9461
Epoch 86/100
1503/1503 [==============================] - 0s 21us/step - loss: 0.6802 - acc: 0.9281
Epoch 87/100
1503/1503 [==============================] - 0s 22us/step - loss: 0.6163 - acc: 0.9261
Epoch 88/100
1503/1503 [==============================] - 0s 21us/step - loss: 0.9383 - acc: 0.8809
Epoch 89/100
1503/1503 [==============================] - 0s 22us/step - loss: 0.6665 - acc: 0.9261
Epoch 90/100
1503/1503 [==============================] - 0s 21us/step - loss: 0.6407 - acc: 0.9261
Epoch 91/100
1503/1503 [==============================] - 0s 23us/step - loss: 0.8513 - acc: 0.8882
Epoch 92/100
1503/1503 [==============================] - 0s 19us/step - loss: 0.7537 - acc: 0.9461
Epoch 93/100


Epoch 1/500
162/162 [==============================] - 0s 102us/step - loss: 1.7002 - acc: 0.6358
Epoch 2/500
162/162 [==============================] - 0s 98us/step - loss: 1.0249 - acc: 0.6605
Epoch 3/500
162/162 [==============================] - 0s 106us/step - loss: 1.6106 - acc: 0.5432
Epoch 4/500
162/162 [==============================] - 0s 111us/step - loss: 0.6992 - acc: 0.8210
Epoch 5/500
162/162 [==============================] - 0s 111us/step - loss: 1.3480 - acc: 0.6173
Epoch 6/500
162/162 [==============================] - 0s 109us/step - loss: 1.3372 - acc: 0.6605
Epoch 7/500
162/162 [==============================] - 0s 103us/step - loss: 2.0288 - acc: 0.5185
Epoch 8/500
162/162 [==============================] - 0s 97us/step - loss: 0.7390 - acc: 0.7840
Epoch 9/500
162/162 [==============================] - 0s 110us/step - loss: 1.0824 - acc: 0.6852
Epoch 10/500
162/162 [==============================] - 0s 105us/step - loss: 1.3023 - acc: 0.6667
Epoch 11/500
162/162 

162/162 [==============================] - 0s 87us/step - loss: 1.5278 - acc: 0.7593
Epoch 167/500
162/162 [==============================] - 0s 103us/step - loss: 0.3021 - acc: 0.8580
Epoch 168/500
162/162 [==============================] - 0s 99us/step - loss: 0.1513 - acc: 0.9568
Epoch 169/500
162/162 [==============================] - 0s 90us/step - loss: 0.2767 - acc: 0.9012
Epoch 170/500
162/162 [==============================] - 0s 106us/step - loss: 1.8444 - acc: 0.6111
Epoch 171/500
162/162 [==============================] - 0s 113us/step - loss: 0.6808 - acc: 0.8086
Epoch 172/500
162/162 [==============================] - 0s 95us/step - loss: 0.9253 - acc: 0.7407
Epoch 173/500
162/162 [==============================] - 0s 94us/step - loss: 0.6461 - acc: 0.8148
Epoch 174/500
162/162 [==============================] - 0s 102us/step - loss: 0.9044 - acc: 0.7716
Epoch 175/500
162/162 [==============================] - 0s 103us/step - loss: 1.0009 - acc: 0.7531
Epoch 176/500
162/1

162/162 [==============================] - 0s 97us/step - loss: 0.7748 - acc: 0.7840
Epoch 331/500
162/162 [==============================] - 0s 99us/step - loss: 1.3790 - acc: 0.7222
Epoch 332/500
162/162 [==============================] - 0s 102us/step - loss: 0.2356 - acc: 0.9074
Epoch 333/500
162/162 [==============================] - 0s 111us/step - loss: 0.8663 - acc: 0.7654
Epoch 334/500
162/162 [==============================] - 0s 115us/step - loss: 0.5179 - acc: 0.8580
Epoch 335/500
162/162 [==============================] - 0s 106us/step - loss: 0.6529 - acc: 0.8519
Epoch 336/500
162/162 [==============================] - 0s 102us/step - loss: 0.2033 - acc: 0.8827
Epoch 337/500
162/162 [==============================] - 0s 111us/step - loss: 1.2402 - acc: 0.7901
Epoch 338/500
162/162 [==============================] - 0s 105us/step - loss: 0.5682 - acc: 0.8457
Epoch 339/500
162/162 [==============================] - 0s 94us/step - loss: 0.9081 - acc: 0.7531
Epoch 340/500
162

162/162 [==============================] - 0s 108us/step - loss: 0.1417 - acc: 0.9444
Epoch 495/500
162/162 [==============================] - 0s 94us/step - loss: 0.4870 - acc: 0.8642
Epoch 496/500
162/162 [==============================] - 0s 106us/step - loss: 0.3178 - acc: 0.9136
Epoch 497/500
162/162 [==============================] - 0s 94us/step - loss: 0.8466 - acc: 0.8272
Epoch 498/500
162/162 [==============================] - 0s 102us/step - loss: 0.3399 - acc: 0.8765
Epoch 499/500
162/162 [==============================] - 0s 103us/step - loss: 0.3172 - acc: 0.8889
Epoch 500/500
162/162 [==============================] - 0s 110us/step - loss: 0.9455 - acc: 0.8704


In [36]:
res

{'knn balanced': {'acc': 0.5345744680851063,
  'dmc': -1035.0,
  'f2': 0.08253094910591471},
 'knn unbalanced': {'acc': 0.9361702127659575, 'dmc': -580.0, 'f2': 0.0},
 'lda balanced': {'acc': 0.8138297872340425,
  'dmc': -235.0,
  'f2': 0.2911392405063291},
 'lda unbalanced': {'acc': 0.9414893617021277,
  'dmc': -545.0,
  'f2': 0.1851851851851852},
 'logistic regression balanced': {'acc': 0.8829787234042553,
  'dmc': -105.0,
  'f2': 0.3951890034364261},
 'logistic regression unbalanced': {'acc': 0.9547872340425532,
  'dmc': -270.0,
  'f2': 0.632183908045977},
 'mlp balanced': {'dmc': -320.0, 'f2': 0.257985257985258},
 'mlp unbalanced': {'dmc': -575.0, 'f2': 0.0},
 'rfc balanced': {'acc': 0.07446808510638298,
  'dmc': -1650.0,
  'f2': 0.0733822548365577},
 'rfc unbalanced': {'acc': 0.9388297872340425, 'dmc': -575.0, 'f2': 0.0},
 'svc balanced': {'acc': 0.07446808510638298,
  'dmc': -1650.0,
  'f2': 0.0733822548365577},
 'svc unbalanced': {'acc': 0.9388297872340425, 'dmc': -575.0, 'f2': 

In [31]:
mlp_unbalanced = model.fit(df_train_unbalanced_x,df_train_unbalanced_y)
res['mlp unbalanced'] = calculate_metrics(df_val_x, df_val_y, model, "keras")


Epoch 1/1
1503/1503 [==============================] - 0s 89us/step - loss: 0.5027 - acc: 0.9115


/home/lukas/Projects/dmc2019/venv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


In [32]:
res

{'knn balanced': {'acc': 0.5345744680851063,
  'dmc': -1035.0,
  'f2': 0.08253094910591471},
 'knn unbalanced': {'acc': 0.9361702127659575, 'dmc': -580.0, 'f2': 0.0},
 'lda balanced': {'acc': 0.8138297872340425,
  'dmc': -235.0,
  'f2': 0.2911392405063291},
 'lda unbalanced': {'acc': 0.9414893617021277,
  'dmc': -545.0,
  'f2': 0.1851851851851852},
 'logistic regression balanced': {'acc': 0.8829787234042553,
  'dmc': -105.0,
  'f2': 0.3951890034364261},
 'logistic regression unbalanced': {'acc': 0.9547872340425532,
  'dmc': -270.0,
  'f2': 0.632183908045977},
 'mlp balanced': {'dmc': -705.0, 'f2': 0.14685314685314685},
 'mlp unbalanced': {'dmc': -575.0, 'f2': 0.0},
 'rfc balanced': {'acc': 0.07446808510638298,
  'dmc': -1650.0,
  'f2': 0.0733822548365577},
 'rfc unbalanced': {'acc': 0.9388297872340425, 'dmc': -575.0, 'f2': 0.0},
 'svc balanced': {'acc': 0.07446808510638298,
  'dmc': -1650.0,
  'f2': 0.0733822548365577},
 'svc unbalanced': {'acc': 0.9388297872340425, 'dmc': -575.0, 'f2'